# Reading Data

In [1]:
# Loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("diabetic_data.csv")
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
df.shape

(101766, 50)

# Data Preprocessing

### Check out the missing values

In [4]:
# Check if there is any duplicate rows.
df[df['encounter_id'].duplicated()]

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted


In [5]:
# Checking for missing values in dataset
# And display only those that are missing values
for col in df.columns:
    if df[col].dtype == object and df[col][df[col] == '?'].count() > 0:
         print(col,df[col][df[col] == '?'].count())

race 2273
weight 98569
payer_code 40256
medical_specialty 49949
diag_1 21
diag_2 358
diag_3 1423


In [6]:
print('gender', df['gender'][df['gender'] == 'Unknown/Invalid'].count()) 

gender 3


In [7]:
missing_weight = df['weight'][df['weight'] == '?'].count()/df.shape[0]*100
print("missing weight", np.round(missing_weight, 2), "%")

missing_payer_code  = df['payer_code'][df['payer_code'] == '?'].count()/df.shape[0]*100
print("missing payer_code ", np.round(missing_payer_code , 2), "%")

missing_medical_specialty  = df['medical_specialty'][df['medical_specialty'] == '?'].count()/df.shape[0]*100
print("missing medical_specialty ", np.round(missing_medical_specialty, 2), "%")

missing weight 96.86 %
missing payer_code  39.56 %
missing medical_specialty  49.08 %


#### Drop features

In [8]:
#dropping columns with large number of missing values
df = df.drop(['weight','payer_code','medical_specialty'], axis = 1)

#observing that all records in citoglipton and examide have the same value => drop
df = df.drop(['citoglipton', 'examide'], axis = 1)

#### Drop records

The columns discharge_disposition_id tells us where the patient went after the hospitalization.
IDs_mapping.csv says that 11,13,14,19,20,21 are related to death or hospice. We should remove these samples from the predictive model.

In [9]:
disposition = pd.DataFrame(df.groupby('discharge_disposition_id').size())
disposition.T

discharge_disposition_id,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,22,23,24,25,27,28
0,60234,2128,13954,815,1184,12902,623,108,21,6,...,14,3691,8,2,1993,412,48,989,5,139


In [10]:
df = df.loc[~df.discharge_disposition_id.isin([11,13,14,19,20,21])]

Fill unknown values then eliminate them

In [11]:
#dropping rows that contain missing values in diag_1-3, race, and gender
df = df.replace('?', np.NaN )
df = df.replace('Unknown/Invalid', np.NaN )
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(11, np.NaN)
df.dropna(inplace=True)
print("The number of dropping records:", 101766 - df.shape[0])

The number of dropping records: 6094


## Feature Engineering

### Numerical Features

The numerical features do not need to modify. I will group all of these features to cols_num variable

In [12]:
numerics = list(set(list(df._get_numeric_data().columns)))
numerics

['number_outpatient',
 'number_diagnoses',
 'admission_type_id',
 'number_emergency',
 'discharge_disposition_id',
 'num_lab_procedures',
 'num_procedures',
 'patient_nbr',
 'time_in_hospital',
 'admission_source_id',
 'num_medications',
 'encounter_id',
 'number_inpatient']

### Categorical Features

Catergorical features are non-numeric data. Therefore, I need to turn them to numerical data which makes sense to the computer.

In [13]:
medications_keys  = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 
        'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 
        'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone','metformin-rosiglitazone', 
        'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']
        
# for col in medications_keys:
#     colname = str(col) + 'temp'
#     df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)
# df['numchange'] = 0
        
# for col in keys:
#     colname = str(col) + 'temp'
#     df['numchange'] = df['numchange'] + df[colname]
#     del df[colname]
    
# df['numchange'].value_counts()  
for col in medications_keys:
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('Steady', 1)
    df[col] = df[col].replace('Up', 1)
    df[col] = df[col].replace('Down', 1)

#### Create a new feature

In [14]:
# combine these 3 features to get fewer features
df['total_service_num'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']

#### Apply re-encoding technique to get fewer categories

In [15]:
# combine Trauma Center(7) and urgent(2) into Emergency(1)
# combine Not Mapped(8) and NULL(6) into Not Available(5)
# Keep the rest
df['admission_type_id'] = df['admission_type_id'].replace(2,1)
df['admission_type_id'] = df['admission_type_id'].replace(7,1)
df['admission_type_id'] = df['admission_type_id'].replace(6,5)
df['admission_type_id'] = df['admission_type_id'].replace(8,5)

# combine categories related to "Discharged to home(1)"
# combine categories related to "Discharged to another hospital(2)"
# combine categories related to "Neonate discharged(10)"
# combine categories related to "NULL(18)"
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([6, 8, 9],1)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([3, 4, 5, 22, 23, 24],2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([12, 15, 16, 17],10)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace([25, 26], 18)

# 
df['admission_source_id'] = df['admission_source_id'].replace([2, 3],1)
df['admission_source_id'] = df['admission_source_id'].replace([5, 6],4)
df['admission_source_id'] = df['admission_source_id'].replace([10, 22, 25],4)
df['admission_source_id'] = df['admission_source_id'].replace([15, 17, 20, 21],9)
df['admission_source_id'] = df['admission_source_id'].replace([13, 14],11)

In [16]:
# Convert categorical features to numerical values
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)

df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)

df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)

In [17]:
# Convert A1Cresult and max_glu_serum to numeric
df['A1Cresult'] = df['A1Cresult'].replace('>7', 1)
df['A1Cresult'] = df['A1Cresult'].replace('>8', 1)
df['A1Cresult'] = df['A1Cresult'].replace('Norm', 0)
df['A1Cresult'] = df['A1Cresult'].replace('None', -99)
df['max_glu_serum'] = df['max_glu_serum'].replace('>200', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('>300', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('Norm', 0)
df['max_glu_serum'] = df['max_glu_serum'].replace('None', -99)

In [18]:
# Convert 10 years interval of Age to integer type
age_dict = {'[0-10)':5, 
            '[10-20)':15, 
            '[20-30)':25, 
            '[30-40)':35, 
            '[40-50)':45, 
            '[50-60)':55, 
            '[60-70)':65, 
            '[70-80)':75, 
            '[80-90)':85,
            '[90-100)':95
            }
df['age'] = df.age.map(age_dict)

In [19]:
# remove the duplicated records with the same patient number, keep the first record to narrow the scope.
# df = df.drop_duplicates(subset= ['patient_nbr'], keep = 'first')
# df.shape

(67128, 46)

In [19]:
# encoding the outcome variable
df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('NO', 0)

In [20]:
df.shape

(95672, 46)

### Encode diag_ into ICD-9

In [21]:
df['level1_diag1'] = df['diag_1']
df['level1_diag2'] = df['diag_2']
df['level1_diag3'] = df['diag_3']

def judgeStrToSpecificValue(colname1,colname2, dataset):
    dataset.loc[dataset[colname1].str.contains('V'), [colname2]] = 0
    dataset.loc[dataset[colname1].str.contains('E'), [colname2]] = 0
    dataset.loc[dataset[colname1] == "?", [colname2]] = 0
    dataset[colname2] = dataset[colname2].astype(float)
    return dataset

df = judgeStrToSpecificValue('diag_1', 'level1_diag1', df)
df = judgeStrToSpecificValue('diag_2', 'level1_diag2', df)
df = judgeStrToSpecificValue('diag_3', 'level1_diag3', df)

# df = df.query("(level1_diag1 >= 250 & level1_diag1 < 251)"
#                  "| (level1_diag2 >= 250 & level1_diag2 < 251)"
#                  "| (level1_diag3 >= 250 & level1_diag3 < 251)"
#                 )

In [22]:
df.shape

(95672, 49)

In [23]:
for index, row in df.iterrows():
    if (row['level1_diag1'] >= 390 and row['level1_diag1'] < 460) or (np.floor(row['level1_diag1']) == 785):
        df.loc[index, 'level1_diag1'] = 1
    elif (row['level1_diag1'] >= 460 and row['level1_diag1'] < 520) or (np.floor(row['level1_diag1']) == 786):
        df.loc[index, 'level1_diag1'] = 2
    elif (row['level1_diag1'] >= 520 and row['level1_diag1'] < 580) or (np.floor(row['level1_diag1']) == 787):
        df.loc[index, 'level1_diag1'] = 3
    elif (np.floor(row['level1_diag1']) == 250):
        df.loc[index, 'level1_diag1'] = 4
    elif (row['level1_diag1'] >= 800 and row['level1_diag1'] < 1000):
        df.loc[index, 'level1_diag1'] = 5
    elif (row['level1_diag1'] >= 710 and row['level1_diag1'] < 740):
        df.loc[index, 'level1_diag1'] = 6
    elif (row['level1_diag1'] >= 580 and row['level1_diag1'] < 630) or (np.floor(row['level1_diag1']) == 788):
        df.loc[index, 'level1_diag1'] = 7
    elif (row['level1_diag1'] >= 140 and row['level1_diag1'] < 240):
        df.loc[index, 'level1_diag1'] = 8
    else:
        df.loc[index, 'level1_diag1'] = 0
        
    if (row['level1_diag2'] >= 390 and row['level1_diag2'] < 460) or (np.floor(row['level1_diag2']) == 785):
        df.loc[index, 'level1_diag2'] = 1
    elif (row['level1_diag2'] >= 460 and row['level1_diag2'] < 520) or (np.floor(row['level1_diag2']) == 786):
        df.loc[index, 'level1_diag2'] = 2
    elif (row['level1_diag2'] >= 520 and row['level1_diag2'] < 580) or (np.floor(row['level1_diag2']) == 787):
        df.loc[index, 'level1_diag2'] = 3
    elif (np.floor(row['level1_diag2']) == 250):
        df.loc[index, 'level1_diag2'] = 4
    elif (row['level1_diag2'] >= 800 and row['level1_diag2'] < 1000):
        df.loc[index, 'level1_diag2'] = 5
    elif (row['level1_diag2'] >= 710 and row['level1_diag2'] < 740):
        df.loc[index, 'level1_diag2'] = 6
    elif (row['level1_diag2'] >= 580 and row['level1_diag2'] < 630) or (np.floor(row['level1_diag2']) == 788):
        df.loc[index, 'level1_diag2'] = 7
    elif (row['level1_diag2'] >= 140 and row['level1_diag2'] < 240):
        df.loc[index, 'level1_diag2'] = 8
    else:
        df.loc[index, 'level1_diag2'] = 0
    
    if (row['level1_diag3'] >= 390 and row['level1_diag3'] < 460) or (np.floor(row['level1_diag3']) == 785):
        df.loc[index, 'level1_diag3'] = 1
    elif (row['level1_diag3'] >= 460 and row['level1_diag3'] < 520) or (np.floor(row['level1_diag3']) == 786):
        df.loc[index, 'level1_diag3'] = 2
    elif (row['level1_diag3'] >= 520 and row['level1_diag3'] < 580) or (np.floor(row['level1_diag3']) == 787):
        df.loc[index, 'level1_diag3'] = 3
    elif (np.floor(row['level1_diag3']) == 250):
        df.loc[index, 'level1_diag3'] = 4
    elif (row['level1_diag3'] >= 800 and row['level1_diag3'] < 1000):
        df.loc[index, 'level1_diag3'] = 5
    elif (row['level1_diag3'] >= 710 and row['level1_diag3'] < 740):
        df.loc[index, 'level1_diag3'] = 6
    elif (row['level1_diag3'] >= 580 and row['level1_diag3'] < 630) or (np.floor(row['level1_diag3']) == 788):
        df.loc[index, 'level1_diag3'] = 7
    elif (row['level1_diag3'] >= 140 and row['level1_diag3'] < 240):
        df.loc[index, 'level1_diag3'] = 8
    else:
        df.loc[index, 'level1_diag3'] = 0

In [24]:
#df.to_csv('./diabetes_data_preprocessed.csv')
df.shape

(95672, 49)

### Dummy coding of all the categorical variables

In [71]:
df = df.drop(['diag_1', 'diag_2', 'diag_3'], axis=1)

In [72]:
data_processed = pd.get_dummies(df, columns=['race', 'gender', 'admission_type_id', 'discharge_disposition_id',
                                      'admission_source_id', 'max_glu_serum', 'A1Cresult', 'level1_diag1',
                                    'level1_diag2', 'level1_diag3', 'change', 'diabetesMed'], drop_first = True)

In [73]:
data_processed.head()

,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,level1_diag3_1.0,level1_diag3_2.0,level1_diag3_3.0,level1_diag3_4.0,level1_diag3_5.0,level1_diag3_6.0,level1_diag3_7.0,level1_diag3_8.0,change_1,diabetesMed_1
1,149190,55629189,15,3,59,0,18,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,64410,86047875,25,2,11,5,13,2,0,1,...,0,0,0,0,0,0,0,0,0,1
3,500364,82442376,35,2,44,1,16,0,0,0,...,1,0,0,0,0,0,0,0,1,1
4,16680,42519267,45,1,51,0,8,0,0,0,...,0,0,0,1,0,0,0,0,1,1
5,35754,82637451,55,3,31,6,16,0,0,0,...,0,0,0,1,0,0,0,0,0,1


### Select featurs from the dataset

In [27]:
column_names = list(df_dummies.columns)

In [28]:
# column_names_removed = ['encounter_id', 'patient_nbr', 'race',
#                         'gender','admission_type_id', 'discharge_disposition_id', 
#                         'admission_source_id', 'number_outpatient','number_emergency',
#                         'diag_1','diag_2','diag_3','max_glu_serum',
#                         'A1Cresult','change','diabetesMed','level1_diag1',
#                         'level1_diag2','level1_diag3','max_glu_serum_-99',
#                         'A1Cresult_-99']

In [32]:
# columns_new = [name for name in column_names if name not in column_names_removed]
# data_processed = df_dummies.loc[:, columns_new]

In [74]:
data_processed.to_csv("./diabetes_data_preprocessed.csv")
data_processed.shape

(95672, 83)

## End Pre-Processing